In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [86]:
import re

In [87]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender = pd.read_csv("../input/titanic/gender_submission.csv")

In [88]:
test_backup = test

In [89]:
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [90]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [91]:
def feature_title(df):
    df['Title'] = df.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.' , x).group(1))
    df['Title'] = df['Title'].replace({'Mlle':'Miss', 'Mme':'Mrs', 'Ms':'Miss'})
    df['Title'] = df['Title'].replace(['Don', 'Dona', 'Rev', 'Dr',
                                                'Major', 'Lady', 'Sir', 'Col', 'Capt', 'Countess', 'Jonkheer'],'Special')
feature_title(train)
feature_title(test)

In [92]:
train['Embarked'] = train['Embarked'].fillna("S")
train['Fam_Size'] = train.SibSp + train.Parch
test['Fam_Size'] = test.SibSp + test.Parch

In [93]:
def fill_age (df) :
    df['Age'] = df.groupby('Title')['Age'].apply(lambda x : x.fillna(x.median()))
    
fill_age(train)
fill_age(test)

In [94]:
test['Fare'] = test.groupby(['Pclass','Embarked' ,'Sex', 'Title'])['Fare'].apply(lambda x: x.fillna(x.median()))

In [95]:
train.columns[train.isna().any()].tolist()

['Cabin']

In [96]:
test.columns[test.isna().any()].tolist()

['Cabin']

In [97]:
y_train = train.Survived

In [98]:
dummy_features = ['Sex','Embarked','Title']
drop_features = ['PassengerId', 'Ticket', 'Name', 'Cabin']
    
train = pd.concat([train, pd.get_dummies(train[dummy_features])], axis = 1, sort = False)
train.drop(columns = train[dummy_features], inplace = True)
train.drop(columns = train[drop_features], inplace = True)

test = pd.concat([test, pd.get_dummies(test[dummy_features])], axis = 1, sort = False)
test.drop(columns = test[dummy_features], inplace = True)
test.drop(columns = test[drop_features], inplace = True)

In [99]:
train.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Fam_Size,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Special
0,0,3,22.0,1,0,7.2500,1,0,1,0,0,1,0,0,1,0,0
1,1,1,38.0,1,0,71.2833,1,1,0,1,0,0,0,0,0,1,0


In [100]:
test.head(2)

,Pclass,Age,SibSp,Parch,Fare,Fam_Size,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Special
0,3,34.5,0,0,7.8292,0,0,1,0,1,0,0,0,1,0,0
1,3,47.0,1,0,7.0000,1,1,0,0,0,1,0,0,0,1,0


In [101]:
train.drop('Survived' , axis = 1 , inplace = True)

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score , confusion_matrix 
import pickle
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

In [104]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LogisticRegression(random_state=0 , max_iter= 50000).fit(train, y_train)
scores = cross_val_score(model, train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)))

Accuracy: 82.722 (0.039)


In [109]:
y_pred = model.predict(test)

In [108]:
y_train_pred = model.predict(train)
confusion_matrix(y_train ,y_train_pred )

array([[484,  65],
       [ 84, 258]])

In [110]:
log_reg = pd.DataFrame({'PassengerId':test_backup['PassengerId'],'Survived':y_pred})

In [111]:
log_reg.to_csv("./submission.csv" , index = False)

**neural network**

In [115]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 3 , 2), random_state=1 , max_iter = 5000)
clf.fit(train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 3, 2), max_iter=5000,
              random_state=1, solver='lbfgs')

In [ ]:
nn_pred_train = clf.predict(train)
confusion_matrix(y_train ,nn_pred_train )

In [117]:
nn_pred = clf.predict(test)

In [ ]:
nn_pred = pd.DataFrame({'PassengerId':test_backup['PassengerId'],'Survived':nn_pred})
nn_pred.to_csv("./submission_nn.csv" , index = False)